In [ ]:
import mytools
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon,Point,LineString,MultiPolygon

# 小区和站点对应关系

In [ ]:
data_z=pd.read_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/2-输出数据/汇总.csv',encoding='gbk')

# 全网小区覆盖区域计算
### --更新热点场景乡镇

In [ ]:
#导入工参数据
gongcan = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/20200712第四版本/1-基础数据/ltejizhan20200616&微站_合并.csv',encoding='gbk'),usecols=[
    '小区CGI(*)','所属地区','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','归属区县','厂家名称' ])
#设置变量通用名
print('设置变量通用名')
id = '小区CGI(*)'
lon = 'RRU经度'
lat = 'RRU纬度'
zhishi = '工作频段'
fw = '方位角'

gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
            (gongcan['RRU经度']<117)&
            (gongcan['RRU纬度']>28)&
            (gongcan['RRU纬度']<34)&
           (gongcan['方位角'].notnull())]

In [ ]:
#只要宏站
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('W-')]
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('R')]
# gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('M-')]

In [ ]:
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan.loc[gongcan['小区中文名'].str.contains('M-')].shape

In [ ]:
#打标场景
gongcan = mytools.gisn.points_coverage_merge(gongcan,lon,lat)
print('添加覆盖距离')
gongcan_loc_juli = mytools.gisn.changjing_distance(gongcan,'区域类型',zhishi)

### 按照覆盖模型修复

In [ ]:
gongcan_loc_juli['工作频段'] = gongcan_loc_juli['工作频段'].apply(lambda x: 900 if x=='FDD900' else 1800)
gd,band= map(np.log10, (gongcan_loc_juli['天线挂高'],gongcan_loc_juli['工作频段']))
gongcan_loc_juli['覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*gd-(-3.81))- 3)/(44.9-6.55*gd))*1000
gongcan_loc_juli['30米覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*np.log10(30)-(-3.81))- 3)/(44.9-6.55*np.log10(30)))*1000
gongcan_loc_juli['使用覆盖半径'] = gongcan_loc_juli['覆盖模型距离'] /gongcan_loc_juli['30米覆盖模型距离']*gongcan_loc_juli['distance']

In [ ]:
def fuck(x):
    if x['区域类型']=='主城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<200:
                return 200
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<150:
                return 150
            elif x['使用覆盖半径']>350:
                return 350
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='县城':
        if x['工作频段']==900:
            if x['使用覆盖半径']<300:
                return 300
            elif x['使用覆盖半径']>430:
                return 430
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<250:
                return 250
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='一般城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1200:
                return 1200
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<350:
                return 350
            elif x['使用覆盖半径']>800:
                return 800
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='乡镇':
        if x['工作频段']==900:
            if x['使用覆盖半径']<430:
                return 430
            elif x['使用覆盖半径']>2200:
                return 2200
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1000:
                return 1000
            else:
                return x['使用覆盖半径']
    elif  x['区域类型']=='农村':
        if x['工作频段']==900:
            if x['使用覆盖半径']<1200:
                return 1200
            elif x['使用覆盖半径']>2500:
                return 2500
            else:
                return x['使用覆盖半径']
        else:
            if x['使用覆盖半径']<800:
                return 800
            elif x['使用覆盖半径']>1500:
                return 1500
            else:
                return x['使用覆盖半径']
    else:
        return 0

In [ ]:
gongcan_loc_juli['distance_use'] = gongcan_loc_juli.apply(fuck,axis=1)

### 按照周边建筑高度修复 

In [ ]:
gongcan_loc_juli['distance_yb'] = gongcan_loc_juli['distance']/2
gongcan_sectors = mytools.gisn.add_sectors_df(gongcan_loc_juli,coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance_yb'],
    has_z=False,sec_col='geometry',shape_dict={'beam': 120, 'per_degree': 10})
f = mytools.othern.file_name_paths('G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/',find='tab')
sjoin_data = gpd.GeoDataFrame()
for name in f:
    tuceng_t = gpd.read_file(name,encoding = 'gbk')
    tuceng_t = tuceng_t.to_crs({'init': 'epsg:4326'})
    linshi_t = gpd.sjoin(gongcan_sectors,tuceng_t)
    linshi_t = linshi_t[[ '小区CGI(*)','建筑物高度','SHAPE_Area']]
    sjoin_data = sjoin_data.append(linshi_t)
    print(name,'ok')

In [ ]:
sjoin_data['volume'] = sjoin_data['建筑物高度'] * sjoin_data['SHAPE_Area']
sjoin_data_volume_h = sjoin_data.groupby('小区CGI(*)').agg({'volume':'sum','SHAPE_Area':'sum'})
sjoin_data_volume_h['h_use'] = sjoin_data_volume_h['volume'] / sjoin_data_volume_h['SHAPE_Area']
sjoin_data_linshi = sjoin_data_volume_h.reset_index()
gongcan_zhangao = gongcan_loc_juli.merge(sjoin_data_linshi[['小区CGI(*)','h_use']],how='left',on='小区CGI(*)')
gongcan_zhangao['高度建筑比'] = (gongcan_zhangao['天线挂高']-gongcan_zhangao['h_use'])/gongcan_zhangao['天线挂高']

In [ ]:
def fuck2(x):
    if (pd.isna(x['distance_use'])) | (x['distance_use']==0 ):
        return x['distance']
    elif pd.isna(x['高度建筑比']):
        return x['distance_use']
    elif x['高度建筑比']>=0.4:
        return x['distance_use']*1.1
    elif x['高度建筑比']>=0.2 and x['高度建筑比']<0.4:
        return x['distance_use']
    elif x['高度建筑比']>=0 and x['高度建筑比']<0.2:
        return x['distance_use']*0.8
    elif x['高度建筑比']<0:
        return x['distance_use']*0.5
    else:
        return x['distance_use']

In [ ]:
gongcan_zhangao['distance_use_3'] = gongcan_zhangao.apply(fuck2,axis=1)
gongcan_zhangao.loc[gongcan_zhangao['distance_use_3'].isna()]
gongcan_zhangao.loc[gongcan_zhangao['小区中文名'].str.contains('M-'),'distance_use_3']=50
gongcan_zhangao.to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/全网覆盖区域修正后.csv',encoding='gbk')
mytools.sound()

In [ ]:
#做扇区
print('做扇区')
gongcan_s = mytools.gisn.add_sectors_df(gongcan_zhangao,coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance_use_3'], has_z=False, sec_col='geometry', shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
gongcan_s['归属区县'] = gongcan_s['归属区县'].fillna('wu')

In [ ]:
gongcan_s.to_pickle(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/全网覆盖区域修正后_扇区.data')

# 去掉农村覆盖到场景边界内的区域

In [ ]:
#---------------------------规划图层缩小50%的距离---------------------------

In [ ]:
guihua = gpd.read_file('G:/1-规划/图层/全省区域规划图层20200110/全省区域规划图层20200110.TAB',encoding='gbk')

In [ ]:
guihua['区域类型2'] = guihua['区域类型'].replace({'3A景区':'农村','4A景区':'农村','5A景区':'农村','景区':'农村','热点':'乡镇'})

In [ ]:
guihua_use = guihua.loc[guihua['区域类型2']!='农村']

In [ ]:
guihua_yiban = mytools.gisn.changjing_distance(guihua_use,'区域类型2','区域类型')

In [ ]:
guihua_yiban['distance2'] = 0-guihua_yiban['distance']/2

In [ ]:
guihua_res = mytools.gisn.buffer_df(guihua_yiban,'distance2')

In [ ]:
guihua_res.columns =['city', 'region', 'name', 'type', 'area', 'geometry', 'type2', 'distance','distance2']

In [ ]:
guihua_res.to_pickle(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/非农村场景缩小.data')

In [ ]:
#---------------------------农村场景去掉非农村场景的部分---------------------------

In [ ]:
gongcan_s_nc = gongcan_s.loc[gongcan_s['区域类型']=='农村']

In [ ]:
gongcan_s_nc['所属地区'] = gongcan_s_nc['所属地区'].fillna('wu')

In [ ]:
gongcan_s_nc = gongcan_s_nc.reset_index(drop=True)

In [ ]:
guihua_res['省份']='hb'

In [ ]:
guihua_res_use = guihua_res.dissolve(by='省份')

In [ ]:
guihua_res_use.shape[0]

In [ ]:
gongcan_s_nc_z = gpd.GeoDataFrame()
for nu in range(0,guihua_res_use.shape[0]-1):
    temp_s = gongcan_s_nc.loc[gongcan_s_nc.index==nu]
    temp_s1 = gpd.overlay(temp_s,guihua_res_use,how='difference')
    gongcan_s_nc_z = gongcan_s_nc_z.append(temp_s1)
    print(nu)

In [ ]:
gongcan_s_fnc = gongcan_s.loc[~(gongcan_s['区域类型']=='农村')]

In [ ]:
gongcan_s_res = gongcan_s_nc_z.append(gongcan_s_fnc)

In [ ]:
mytools.sound()

In [ ]:
gongcan_s_res.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/修正农村后的单小区覆盖.data')

# 小区单独覆盖区域计算
### --跟新农村到非农村场景的修正

In [ ]:
gongcan_s = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/修正农村后的单小区覆盖.data')

In [ ]:
gongcan_s = gongcan_s[['小区CGI(*)', '小区中文名', '所属地区', 'RRU经度', 'RRU纬度', '天线挂高', '方位角', '工作频段',
       '归属区县', '厂家名称', '区域类型','geometry']]


In [ ]:
gongcan_s = gongcan_s.loc[gongcan_s['归属区县']=='东湖高新开发区']

In [ ]:
gongcan_s.shape

In [ ]:
def fc(x):
    lon = x['RRU经度']
    lat = x['RRU纬度']
    #筛选范围内的所有扇区
    gongcan_s_temp = gongcan_s.loc[(gongcan_s['RRU经度']>(lon-0.05) )&(gongcan_s['RRU经度']<(lon+0.05) ) & 
                  (gongcan_s['RRU纬度']>(lat-0.05) )&(gongcan_s['RRU纬度']<(lat+0.05))]
    #剔除掉共站址的扇区
    b = data_z.loc[data_z['小区名']==x['小区中文名'],'站点名称']
    if len(list(b.values)) == 0 :
        geometry='无效数据'
    else:
        use_s = gongcan_s_temp.loc[~gongcan_s_temp['小区中文名'].isin(list(data_z.loc[data_z['站点名称']==b.values[0],'小区名'].values))]
        if use_s.shape[0]<1:
            geometry='孤站'
        else:
            use_s['省份'] = 'hb'
            use_s1 = use_s.dissolve(by = '省份')
            use_s1['geometry'] = use_s1.buffer(0.0000001)
            geometry = x['geometry'].difference(use_s1['geometry'][0])
            if geometry.area==0:
                geometry=1
    return geometry

In [ ]:
gongcan_s['ji_geo'] = gongcan_s.apply(fc,axis=1)#gongcan_s.iloc[36160:36177,]

In [ ]:
mytools.sound()

# 面积计算

In [ ]:
gongcan_s_area = mytools.gisn.add_area(gongcan_s,'自身面积')

In [ ]:
gongcan_s_area_1 = gongcan_s_area.loc[~((gongcan_s_area['ji_geo']==1) | (gongcan_s_area['ji_geo']=='无效数据') | (gongcan_s_area['ji_geo']=='孤站')) ]

In [ ]:
gongcan_s_area_2 = gongcan_s_area.loc[(gongcan_s_area['ji_geo']==1) | (gongcan_s_area['ji_geo']=='无效数据') | (gongcan_s_area['ji_geo']=='孤站') ]

In [ ]:
gongcan_s_area_2['单独覆盖区域面积'] = gongcan_s_area_2['ji_geo']

In [ ]:
gongcan_s_area_1 = gongcan_s_area_1.set_geometry('ji_geo')

In [ ]:
gongcan_s_area_1 = mytools.gisn.add_area(gongcan_s_area_1,'单独覆盖区域面积')

In [ ]:
res = gongcan_s_area_1.append(gongcan_s_area_2)

In [ ]:
res.drop(columns=['geometry','ji_geo']).to_csv(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/修正农村后的每个小区单独覆盖面积.csv',encoding='gbk',index=False)

In [ ]:
res.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/20200812重构/1-全网小区半径修正后-更新了热点为乡镇/修正农村后的每个小区单独覆盖面积.data')

# 其他计算

In [ ]:
zm_xd = pd.read_excel(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/临时需求，将主城区一般城区与农村小区覆盖有交集的删掉/东新小区覆盖为0.xlsx',
    encoding='gbk')

In [ ]:
zm_xd.shape

In [ ]:
zm_xd_use = zm_xd.merge(gongcan_s[['小区CGI(*)','小区中文名','geometry']],how='left',on=['小区CGI(*)','小区中文名'])

In [ ]:
gongcan_nc = gongcan_s.loc[gongcan_s['区域类型']=='农村']

In [ ]:
gongcan_nc = gongcan_nc.dissolve(by='所属地区')

In [ ]:
zm_xd_use_gdf = gpd.GeoDataFrame(zm_xd_use,crs={"init": "epsg:4326"} ,geometry='geometry')

In [ ]:
res = gpd.sjoin(zm_xd_use_gdf,gongcan_nc)

In [ ]:
res.drop(columns='geometry').to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/临时需求，将主城区一般城区与农村小区覆盖有交集的删掉/res.csv',encoding='gbk')